# Info
- Overleaf document: https://www.overleaf.com/read/ggqqbccgrqbg#eff08f
- Python version: Python 3.13.0 (tags/v3.13.0:60403a5, Oct  7 2024, 09:38:07) [MSC v.1941 64 bit (AMD64)] on win32
- Structure:
     - process data (verbose and clunky, but whatever)
     - define main grid pricing (pulled from Hydro)
     - define shared battery specs (pulled from paper)
     - define broadcast info (WIP)
     - define classes for coordinator and households (WIP)
     - main program/algo (WIP)

### Household data dataframe
| `timestamp` | `g_pv` | `price` | `D_max` | `D_min` | `alpha` | `g_l` | `g_s` | `g_ch` | `g_dis` | `D` |
|-------------|--------|---------|---------|---------|---------|-------|-------|--------|---------|-----|
| string | float | float | float | float | float | float | float | float | float |float|
|raw timestamp|solar panel generation|grid electricity unit cost|desired energy (all appliances)|non-sheddable energy (fridge, freezer)|load-shedding sensitivity|optimized energy from grid to household|optimized energy from grid to battery|optimized PV energy to battery|optimized energy from battery to household|optimized actual load|

### Main grid pricing data
Use BC Hydro's residential time-of-day pricing scheme (https://app.bchydro.com/accounts-billing/rates-energy-use/electricity-rates/residential-rates/tiered-time-of-day.html), forgoing the daily surcharge for simplicity:
| Description | Time | \$/kWh  |
|-------------|------|---------|
| Off-peak    | $[07:00, 16:00) \cup [21:00, 23:00)$ | 0.1097 |
| On-peak     | $[16:00, 21:00)$                     | 0.1597 |
| Overnight   | $[23:00, 07:00)$                     | 0.0597 |

The unit price of main grid energy can be generated for a given timestamp using a function.

In [441]:
import pandas as pd
import numpy as np
from datetime import datetime
from cvxopt import matrix, solvers

In [442]:
# columns we want to use
relevant_cols = ['cet_cest_timestamp',
                 
                 'DE_KN_residential1_dishwasher', 'DE_KN_residential1_freezer', 'DE_KN_residential1_heat_pump',
                 'DE_KN_residential1_pv','DE_KN_residential1_washing_machine',
                 
                 'DE_KN_residential3_circulation_pump', 'DE_KN_residential3_dishwasher', 'DE_KN_residential3_freezer',
                 'DE_KN_residential3_pv', 'DE_KN_residential3_refrigerator', 'DE_KN_residential3_washing_machine',
                 
                 'DE_KN_residential4_dishwasher', 'DE_KN_residential4_ev', 'DE_KN_residential4_freezer',
                 'DE_KN_residential4_heat_pump', 'DE_KN_residential4_pv', 'DE_KN_residential4_refrigerator',
                 'DE_KN_residential4_washing_machine',
                 
                 'DE_KN_residential6_circulation_pump', 'DE_KN_residential6_dishwasher', 'DE_KN_residential6_freezer',
                 'DE_KN_residential6_grid_import', 'DE_KN_residential6_pv', 'DE_KN_residential6_washing_machine']

# extract the columns we want to use and drop any rows containing NaN cells
raw_household_data = pd.read_csv('opsd-household_data-2020-04-15/household_data_60min_singleindex.csv', usecols=relevant_cols).dropna()

In [443]:
# calculate discrete difference to get consumption for time slot (data is cumulative)
raw_household_data[relevant_cols[1:]] = raw_household_data[relevant_cols[1:]].diff()

# drop initial NaN row that results from the discrete difference
raw_household_data = raw_household_data.dropna()

In [444]:
# main grid pricing
prices = {'p_max': 0.1597,
          'p_normal': 0.1097,
          'p_min': 0.0597}
def price(current_timestamp):
    # BC Hydro's time-of-day pricing scheme w/o daily surcharge
    # current_timestamp is a string with the following format
    # YYYY-MM-DDTHH:MM:SS+ZZZZ
    # e.g. 2014-12-11T18:00:00+0100
    ts = datetime.strptime(current_timestamp, '%Y-%m-%dT%H:%M:%S%z')
    if ((ts.hour >= 7) and (ts.hour < 16)) or ((ts.hour >= 21) and (ts.hour < 23)):
        return prices['p_normal'] # off-peak: [7,16) u [21,23)
    elif (ts.hour >= 16) and (ts.hour < 21):
        return prices['p_max'] # on-peak: [16,21)
    else:
        return prices['p_min'] # overnight

In [445]:
# generate price column
raw_household_data['price'] = raw_household_data['cet_cest_timestamp'].apply(price)

In [446]:
# create a dataframe for each household
# shorten timestamp header
# remove "DE_KN_residential<number>_" from headers
# rename "DE_KN_residentialx_pv" to "g_pv"
df_1 = raw_household_data[['cet_cest_timestamp',
                           'DE_KN_residential1_dishwasher',
                           'DE_KN_residential1_freezer',
                           'DE_KN_residential1_heat_pump',
                           'DE_KN_residential1_pv',
                           'DE_KN_residential1_washing_machine',
                           'price']].copy()
df_1 = df_1.rename(columns={'cet_cest_timestamp': 'timestamp',
                            'DE_KN_residential1_dishwasher': 'dishwasher',
                            'DE_KN_residential1_freezer': 'freezer',
                            'DE_KN_residential1_heat_pump': 'heat_pump',
                            'DE_KN_residential1_pv': 'g_pv',
                            'DE_KN_residential1_washing_machine': 'washing_machine'})
df_2 = raw_household_data[['cet_cest_timestamp',
                           'DE_KN_residential3_circulation_pump',
                           'DE_KN_residential3_dishwasher',
                           'DE_KN_residential3_freezer',
                           'DE_KN_residential3_pv',
                           'DE_KN_residential3_refrigerator',
                           'DE_KN_residential3_washing_machine',
                           'price']].copy()
df_2 = df_2.rename(columns={'cet_cest_timestamp': 'timestamp',
                            'DE_KN_residential3_circulation_pump': 'circulation_pump',
                            'DE_KN_residential3_dishwasher': 'dishwasher',
                            'DE_KN_residential3_freezer': 'freezer',
                            'DE_KN_residential3_pv': 'g_pv',
                            'DE_KN_residential3_refrigerator': 'refrigerator',
                            'DE_KN_residential3_washing_machine': 'washing_machine'})
df_3 = raw_household_data[['cet_cest_timestamp',
                           'DE_KN_residential4_dishwasher',
                           'DE_KN_residential4_ev',
                           'DE_KN_residential4_freezer',
                           'DE_KN_residential4_heat_pump',
                           'DE_KN_residential4_pv',
                           'DE_KN_residential4_refrigerator',
                           'DE_KN_residential4_washing_machine',
                           'price']].copy()
df_3 = df_3.rename(columns={'cet_cest_timestamp': 'timestamp',
                            'DE_KN_residential4_dishwasher': 'dishwasher',
                            'DE_KN_residential4_ev': 'ev',
                            'DE_KN_residential4_freezer': 'freezer',
                            'DE_KN_residential4_heat_pump': 'heat_pump',
                            'DE_KN_residential4_pv': 'g_pv',
                            'DE_KN_residential4_refrigerator': 'refrigerator',
                            'DE_KN_residential4_washing_machine': 'washing_machine'})
df_4 = raw_household_data[['cet_cest_timestamp',
                           'DE_KN_residential6_circulation_pump',
                           'DE_KN_residential6_dishwasher',
                           'DE_KN_residential6_freezer',
                           'DE_KN_residential6_pv',
                           'DE_KN_residential6_washing_machine',
                           'price']].copy()
df_4 = df_4.rename(columns={'cet_cest_timestamp': 'timestamp',
                            'DE_KN_residential6_circulation_pump': 'circulation_pump',
                            'DE_KN_residential6_dishwasher': 'dishwasher',
                            'DE_KN_residential6_freezer': 'freezer',
                            'DE_KN_residential6_pv': 'g_pv',
                            'DE_KN_residential6_washing_machine': 'washing_machine'})

In [447]:
# compute D_max and D_min for each household
# D_max is sum of all appliances
# D_min is sum of refrigerator and freezer
def compute_Ds(dataframe):
    D_max_cols = []
    D_min_cols = []
    for label in dataframe.columns:
        if label not in ['timestamp', 'g_pv']:
            D_max_cols.append(label)
            if label in ['freezer', 'refrigerator']:
                D_min_cols.append(label)
    dataframe['D_max'] = dataframe[D_max_cols].sum(axis=1)
    dataframe['D_min'] = dataframe[D_min_cols].sum(axis=1)
    return dataframe
df_1 = compute_Ds(df_1)
df_2 = compute_Ds(df_2)
df_3 = compute_Ds(df_3)
df_4 = compute_Ds(df_4)

In [448]:
# delete appliance columns
df_1 = df_1.drop(columns=['dishwasher', 'freezer', 'heat_pump', 'washing_machine'])
df_2 = df_2.drop(columns=['circulation_pump', 'dishwasher', 'freezer', 'refrigerator', 'washing_machine'])
df_3 = df_3.drop(columns=['dishwasher', 'ev', 'freezer', 'heat_pump', 'refrigerator', 'washing_machine'])
df_4 = df_4.drop(columns=['circulation_pump', 'dishwasher', 'freezer', 'washing_machine'])

In [449]:
# generate alpha columns
df_1['alpha'] = np.random.default_rng(10).integers(low=15, high=35, size=len(df_1), endpoint=True).astype('f') / 10
df_2['alpha'] = np.random.default_rng(20).integers(low=15, high=35, size=len(df_2), endpoint=True).astype('f') / 10
df_3['alpha'] = np.random.default_rng(30).integers(low=15, high=35, size=len(df_3), endpoint=True).astype('f') / 10
df_4['alpha'] = np.random.default_rng(40).integers(low=15, high=35, size=len(df_4), endpoint=True).astype('f') / 10

In [450]:
# add empty columns for g_l, g_s, g_ch, g_dis, and D
df_1[['g_l','g_s','g_ch','g_dis','D']] = None
df_2[['g_l','g_s','g_ch','g_dis','D']] = None
df_3[['g_l','g_s','g_ch','g_dis','D']] = None
df_4[['g_l','g_s','g_ch','g_dis','D']] = None

In [451]:
# dict to store battery parameters (using numbers from paper)
battery = {'S_cap': 80.0,
           'S_max': 80.0,
           'S_min': 0.2*80.0,
           'eta_ch': 0.8,
           'eta_dis': 1.25,
           'R_ch': 0.15*80.0,
           'R_dis': 0.15*80.0,
           's_0': 80.0}

In [452]:
# dict to store broadcast values - coordinator writes, households read
broadcast = {'slot': 0,
             'repeat': False,
             'V': None,
             'K_b': None}

In [463]:
# coordinator class
class Coordinator:
    def __init__(self, num_households):
        self.V = battery['eta_ch'] \
                * (battery['S_max'] \
                   -battery['S_min'] \
                   -battery['eta_ch']*battery['R_ch'] \
                   -battery['eta_dis']*battery['R_dis']) \
                / (prices['p_max']-prices['p_min'])
        self.K_b = battery['s_0']-battery['S_min']-battery['eta_dis']*battery['R_dis']-self.V*prices['p_max']
        self.Con = [1/num_households for _ in range(num_households)]

    def update_Con(self, household_index, household_rec):
        self.Con[household_index] = self.Con[household_index] \
                                   + battery['eta_ch']*(household_rec['g_ch']+household_rec['g_s']) \
                                   - battery['eta_dis']*household_rec['g_dis']

# household class
class Household:
    # class variables - initial values are common across all households
    rec = None # row of data for current time slot
    H_l = 0 # load queue
    has_surplus = False # flag for energy surplus
    k = 1 # optimization counter
    xi_ch = 1 # portion of R_ch taken
    xi_dis = 1 # portion of R_dis taken

    def __init__(self, df, beta):
        self.df = df # dataframe containing household data
        self.beta = beta # maximum load-shedding ratio

    def update_has_surplus(self):
        if (self.rec['g_pv'].item()-self.rec['D_max'].item()) >= 0:
            self.has_surplus = True
        else:
            self.has_surplus = False

    def solve_a(self):
        Q = matrix([[2*broadcast['V']*self.rec['alpha']],
                    [0.0,0.0,0.0,0.0],
                    [0.0,0.0,0.0,0.0],
                    [0.0,0.0,0.0,0.0]])
        p = matrix([-(H_l/(self.rec['D_max']-self.rec['D_min']))-2*broadcast['V']*self.rec['alpha']*self.rec['D_max'],
                    broadcast['K_b'],
                    broadcast['V']*self.rec['price'],
                    broadcast['V']*self.rec['price']])
        G = matrix([[1.0,-1.0,1.0,0.0,0.0],
                    [0.0,0.0,1.0,1.0,-1.0],
                    [0.0,0.0,0.0,0.0,0.0],
                    [0.0,0.0,0.0,1.0,-1.0]])
        h = matrix([self.rec['D_max'],
                    -self.rec['D_min'],
                    self.rec['g_pv'],
                    self.xi_ch*battery['R_ch'],
                    0.0])
        sol=solvers.qp(Q, p, G, h)
        x = sol['x']

    def solve_b(self):
        Q = matrix([[2*broadcast['V']*self.rec['alpha']],
                    [0.0,0.0,0.0,0.0],
                    [0.0,0.0,0.0,0.0],
                    [0.0,0.0,0.0,0.0]])
        p = matrix([-(H_l/(self.rec['D_max']-self.rec['D_min']))-2*broadcast['V']*self.rec['alpha']*self.rec['D_max'],
                    -broadcast['K_b'],
                    broadcast['V']*self.rec['price'],
                    broadcast['V']*self.rec['price']])
        G = matrix([[1.0,-1.0,0.0,0.0],
                    [0.0,0.0,1.0,1.0],
                    [0.0,0.0,0.0,0.0],
                    [0.0,0.0,0.0,0.0]])
        h = matrix([self.rec['D_max'],
                    -self.rec['D_min'],
                    self.xi_dis*battery['R_dis'],
                    0.0])
        A = matrix([1.0,-1.0,-1.0,0.0], (1,4))
        b = matrix(self.rec['g_pv'])
        sol=solvers.qp(Q, p, G, h, A, b)
        x = sol['x']
	
    def update_H_l(self):
        D_max = self.rec['D_max'].item()
        D_min = self.rec['D_min'].item()
        D = self.rec['D'].item()
        self.H_l = max(self.H_l-self.beta,0) + ((D_max-D)/(D_max-D_min))

# create coordinator object
coordinator = Coordinator(4)

# create household objects
rng = np.random.default_rng(seed=42) # seed for beta values randomly generated in [0.5,0.7]
betas = rng.integers(low=50, high=70, size=4, endpoint=True) / 100
household_1 = Household(df_1, betas[0].item())
household_2 = Household(df_2, betas[1].item())
household_3 = Household(df_3, betas[2].item())
household_4 = Household(df_4, betas[3].item())